### Import and install necessary Python packages

In [1]:
import pandas as pd
import datetime
import sys
#!{sys.executable} -m pip install plotly

### Re-usable functions

In [2]:
'''
Print columns with null value.
'''
def get_cols_with_null(df):
    # Get a list containing each column and number of null values in each column
    col_list = []
    x = len(df) - df.count()
    for i in range(0, len(all_columns)):
        if x[i] > 0:
            col_list.append(str(all_columns[i]))
            
    return col_list

In [3]:
'''
Transpose the count values for each of the day columns (75 of them) into a a 3 column dataframe containing the id, day value and the count for the day.
'''
def trans_data(input_df, date_col_names):
    col_id =   []
    col_date = []
    col_cnt =  []
    ncol = len(date_col_names)
    for i in range(0, len(input_df)):
        for j in range(0, ncol-1):
          col_id.append(input_df.iloc[i][75])
          col_date.append(date_col_names[j])
          col_cnt.append(input_df.iloc[i][j])

    the_dict = {'id':col_id,'the_date':col_date, 'the_count':col_cnt}
    out_df = pd.DataFrame(the_dict) 
    return out_df

In [4]:
'''
Generate consistent column names for each day that could be later used as valid dates. 
The column names for dates is used for all data sets.
'''
start_date = datetime.datetime(2020, 1, 22)
end_date = datetime.datetime(2020, 4, 5)
oneDay = datetime.timedelta(days=1)

# Generate column names for days to cover
date_col_names = []
while start_date <= end_date:
    col_name = "{}-{}-{}".format(start_date.day, start_date.month, start_date.year)
    start_date += oneDay
    date_col_names.append(col_name)

In [5]:
'''
Read the 3 data sets into separate dataframes and perform initial data wrangling
'''
c1 = pd.read_csv("E:\\Personal_Files\\Dariush\monash\\visual\\AS3\\new\\time_series_covid19_confirmed_global.csv")
#c1 = pd.read_csv("E:\\Personal_Files\\Dariush\monash\\visual\\AS3\\new\\reduced.csv")
d1 = pd.read_csv("E:\\Personal_Files\\Dariush\monash\\visual\\AS3\\new\\time_series_covid19_deaths_global.csv")
r1 = pd.read_csv("E:\\Personal_Files\\Dariush\monash\\visual\\AS3\\new\\time_series_covid19_recovered_global.csv")

### Data Wrangling

In [6]:
'''
1. Rename date columns
2. identify columns with a null vale.
3. State happens to be the only one; Replace its missing values with the name of the country;
4. Add an 'id' column with a sequential number (equalling the index.
'''
all_columns = ['State', 'Country', 'Lat', 'Long'] + date_col_names
c1.columns = all_columns
print("Before update: Names of columns with null values: ", get_cols_with_null(c1))
c1.State.fillna(c1.Country, inplace=True)
print("After update: Names of columns with null values: ", get_cols_with_null(c1))
c1['id'] = c1.index 

Before update: Names of columns with null values:  ['State']
After update: Names of columns with null values:  []


In [7]:
'''
5. Create two subsets of the list. 
One that includes all columns to and including longitude, 
and another that includes from the first date column to id.
'''
c11 = pd.concat((c1.loc[:, :'Long'], c1.loc[:, 'id']), axis=1)
c12 = c1.loc[:, '22-1-2020':'id']  

In [8]:
'''
6. Rename the columns as we want to make room for same column names to hold the increments. 
Figures/counts as given are accummulative. 
We need increments to be able to perform aggregation at the time of visualization.
'''
for ix2 in range(0, len(date_col_names)): 
    col_name = 'X-' + date_col_names[ix2]
    c12[col_name] = 0

In [9]:
'''
7. Extract increments by subtracting the next day from its previous day.
'''
for ix1 in range(0, len(c12)):
    for ix2 in range(0, len(date_col_names)-1): 
        new_col = 'X-' + date_col_names[ix2+1]
        delta = c12.iloc[ix1][ix2+1] - c12.iloc[ix1][ix2]
        new_col_no = len(date_col_names) + ix2
        c12.loc[ix1].at[new_col] = delta
        #print(new_col, c12.iloc[ix1][ix2+1], c12.iloc[ix1][ix2], delta)
        
# c12.to_csv (r'E:\Personal_Files\Dariush\monash\visual\AS3\\new\temp_condf.csv', index = False, header=True)

In [11]:
'''
8. Now, rename the columns back to what they were.
'''
last_day = 'X-' + date_col_names[len(date_col_names)-1]
c12a = c12.loc[:, 'id':last_day]

for ix in range(0, len(date_col_names)): 
    old_col_name = 'X-' + date_col_names[ix]
    new_col_name = date_col_names[ix]
    c12a.rename(columns={old_col_name:new_col_name}, inplace=True)

### Perform same data wrangling for second data set.

In [ ]:
d1.columns = all_columns
print("Before update: Names of columns with null values: ", get_cols_with_null(d1))
d1.State.fillna(d1.Country, inplace=True)
print("After update: Names of columns with null values: ", get_cols_with_null(d1))
d1['id'] = d1.index 

In [ ]:
d11 = pd.concat((d1.loc[:, :'Long'], d1.loc[:, 'id']), axis=1)
d12 = d1.loc[:, '22-1-2020':'id']

d12h_frame = trans_data(d12, date_col_names)
d12h_frame = d12h_frame.rename(columns={'the_date': 'death_date', 'the_count': 'death_count'})
df_deaths = pd.merge(d11, d12h_frame,on='id')
if len(df_confirmed) == len(d1) * (len(date_col_names) - 1):
   print("Success: The number of rows in the resulting data frame matches the number of rows \
in the original data frame by the number of data")
else:
   print("Something wrong: The number of rows in the resulting data frame does not matches with \
number of rows in the original data frame by the number of data")

In [ ]:
'''
import plotly.express as px

fig = px.bar(df_deaths, x='death_date', y='death_count',
              color='death_date',
             labels={'Confirmed Cases':'Corona Stats'}, height=400)
fig.show()
'''

### Perform same data wrangling process for recovered data set.

In [ ]:
r1.columns = all_columns
print("Before update: Names of columns with null values: ", get_cols_with_null(r1))
r1.State.fillna(r1.Country, inplace=True)
print("After update: Names of columns with null values: ", get_cols_with_null(r1))
r1['id'] = r1.index 

In [ ]:
r11 = pd.concat((r1.loc[:, :'Long'], r1.loc[:, 'id']), axis=1)
r12 = r1.loc[:, '22-1-2020':'id']

r12h_frame = trans_data(r12, date_col_names)
r12h_frame = r12h_frame.rename(columns={'the_date': 'recovered_date', 'the_count': 'recovered_count'})
df_recovered = pd.merge(r11, r12h_frame,on='id')
if len(df_recovered) == len(r1) * (len(date_col_names) - 1):
   print("Success: The number of rows in the resulting data frame matches the number of rows \
in the original data frame by the number of data")
else:
   print("Something wrong: The number of rows in the resulting data frame does not matches with \
number of rows in the original data frame by the number of data")

In [ ]:
'''
import plotly.express as px

fig = px.bar(df_recovered, x='recovered_date', y='recovered_count',
              color='recovered_date',
             labels={'Recovered Cases':'Corona Stats'}, height=400)
fig.show()
'''

In [ ]:
df_confirmed_death = pd.merge(df_confirmed, df_deaths,  how='left', left_on=['Country', 'State', 'confirmed_date'], right_on=['Country', 'State', 'death_date'])
df_confirmed_death_recovered = pd.merge(df_confirmed_death, df_recovered, how='left', left_on=['Country', 'State', 'confirmed_date'], right_on=['Country', 'State', 'recovered_date'])

In [ ]:
len(df_confirmed_death_recovered)

In [ ]:
df_final = df_confirmed_death_recovered[['Country', 'State', 'Long', 'Lat', 'confirmed_date', 'confirmed_count', 'death_count', 'recovered_count']]

In [ ]:
df_final.head(10)

In [ ]:
df_final.to_csv (r'E:\Personal_Files\Dariush\monash\visual\AS3\\new\time_series_covid19_ALL_global.csv', index = False, header=True)